# **Transformer Interpretability**

In this coding homework, you will:
* Implement a single attention head
* Implement an induction copy head by combining a previous token head with a copying head.

In [ ]:
#pull the github repo
!git clone https://github.com/Berkeley-CS182/cs182fa25_public.git

In [ ]:
%cd cs182fa25_public/hw11/code/q_coding_interpretability/

## **Imports & Preliminaries**

In [1]:
import numpy as np
import json

seed = 2025
eps = 1e-6
np.random.seed(seed)

## **Question 1: A Single Attention Head**
In this question, you'll implement an attenton head.

### Specifications:
* The query-key matrices are provided already multiplied together (i.e., we provide as input $W_{QK} = W_Q^\top W_K$, called `WQK`).
* The output-value matrices are provided already multiplied together (i.e., we provide as input $W_{OV} = W_O^\top W_V$, called `WOV`).
* Attention inputs are also provided as a list of `d_model`-length vectors, called `attn_input`. The list elements correspond to positions in the context.
* The desired outputs are the outputs the attention head produces at each position. This should be a list of `d-model`-length vectors of the same length as the input.
* **Causal masking:** When implementing attention, mask out positions that come after the current position by setting their attention scores to negative infinity (e.g., `-1e9`) before applying softmax.
* You should first convert the inputs to numpy arrays as a first step.

### Note:
* You should **not** use `np.softmax` when calculating the attention scores.

In [2]:
def single_attention_head(attn_input, WQK, WOV):
  """
  Implement a single attention head

  Args:
      attn_input: List[List[float]] - Input to attention head, shape (seq_len, d_model)
      WQK: List[List[float]] - Premultiplied query-key matrix, shape (d_model, d_model)
      WOV: List[List[flot]] - Premultiplied output-value matrix, shape (d_model d_model)

  Returns:
      numpy.ndarray - Output of attention head, shape (seq_len, d_model)
  """
  # step 0: convert to numpy arrays
  X = np.asarray(attn_input)
  WQK, WOV = np.asarray(WQK), np.asarray(WOV)

  seq_len, d_model = X.shape

  # step 1: get pre-attention scores
  Z = X @ WQK @ X.T

  # step 2: apply causal mask
  mask = np.triu(np.ones((seq_len, seq_len), dtype=bool), k = 1)
  Z[mask] = -1e9

  # step 3: apply softmax
  e_Z = np.exp(Z)
  attn_scores = e_Z / np.sum(e_Z, axis=-1, keepdims=True)  # IMPORTANT: keepdims=True

  # step 4: multiply attention input by output-value matrix
  XOV = X @ WOV.T

  # step 5: multiply attention scores by X_OV
  out = attn_scores @ XOV

  return out

### Test `single_attention_head` basic

In [3]:
attn_input = [[0, 1],
              [1, 1],
              [1, 2]]
WQK = [[1, 1],
       [0, 0]]
WOV = [[1, 1],
       [0, 0]]

expected_out = [[1.0, 0.0],
                [1.7310585786300048, 0.0],
                [2.5752103826044417, 0.0]]

out = single_attention_head(attn_input, WQK, WOV).tolist()

assert np.isclose(expected_out, out, atol=eps).all(), f"Failed:\nExpected: {expected_out}\nGot: {out}"
print("Test case passed ✅")

Test case passed ✅


### Test `single_attention_head`

In [5]:
with open('single_attention_head_test_cases.json', 'r') as f:
  single_attention_head_test_cases = json.load(f)

for test_case_id in single_attention_head_test_cases:
  attn_input, WQK, WOV, expected_out = single_attention_head_test_cases[test_case_id]
  out = single_attention_head(attn_input, WQK, WOV).tolist()
  id = test_case_id.split(" ")[-1]
  assert np.isclose(expected_out, out, atol=eps).all(), f"Test Case {id} Failed:\nExpected: {expected_out}\nGot: {out}"

print("All test cases passed ✅")

All test cases passed ✅


In [ ]:
# # Generating test cases
# d_models = [9, 18, 26, 34, 45, 80, 100]
# seq_lens = [5, 13, 23, 29, 38, 65, 84]

# i = 1
# out = {}

# for d_model in d_models:
#   for seq_len in seq_lens:
#     attn_input = np.random.normal(size=(seq_len, d_model)).tolist()
#     WQK = np.random.normal(size=(d_model, d_model)).tolist()
#     WOV = np.random.normal(size=(d_model, d_model)).tolist()
#     out[f"Test case {i}"] = [attn_input, WQK, WOV, single_attention_head(attn_input, WQK, WOV).tolist()]
#     i += 1
# # 
# with open('single_attention_head_test_cases.json', 'w') as f:
#   json.dump(out, f, indent=2)

## **Question 2: An Induction Copy Head**
In this problem, you will implement an induction copy head by combining a previous token head with a copying head

### Background:
An induction head operates by predicting that previously-seen adjacencies in the sequence will be seen again. That is, it predicts `ab...a` will be followed by `b` (or any `a,b`).

### Specifications:
* #### Vocabulary and Embeddings
  - Vocabulary size: 4
  - Tokens: `a`, `b`, `c`, `d`
  - Maximum sequence length: 5
  - Embedding: 2-hot encoded with `d_model = 9`
    - First 4 dimensions: 1-hot encoding of token vocabulary
      - `a` -> [1, 0, 0, 0, ...]
      - `b` -> [0, 1, 0, 0, ...]
      - `c` -> [0, 0, 1, 0, ...]
      - `d` -> [0, 0, 0, 1, ...]
    - Next 5 dimensions: 1-hot encoding of position
      - Position 0 -> [..., 1, 0, 0, 0, 0]
      - Position 1 -> [..., 0, 1, 0, 0, 0]
      - Position 2 -> [..., 0, 0, 1, 0, 0]
      - Position 3 -> [..., 0, 0, 0, 1, 0]
      - Position 4 -> [..., 0, 0, 0, 0, 1]
  - The unembeddings are the same as the embeddings. The model produces an output vector of length 4 which encodes the logits on `a,b,c,d` respectively.
* ### Induction Head Mechanism
  - Your implementation should consist of two stages.
     1. **Previous Token Head:** Identifies tokens that are directly adjacent to each other (using position information)
     2. **Induction Head:** Takes the output from the previous token head and copies the token that follows matching patterns.

     Together, these implement the pattern: `a,b,...,a -> b` for all a,b.
* ### Previous Token Head
  - Attend from each position to its direct predecessor
  - Extract and pass forward the token identities
  - Use `attention_strength` for the non-zero entries in the QK matrices.
  - Use only 0's and 1's in the OV matrices.
* ### Induction Head
  - Take input from the previous token head summed with the original token embedding. To make life simpler:
     - Exclude the positional embedding from this sum! That is, just sum the non-positional part of the input embedding with the output from the previous token head.
     - Delete from this sum the first token position, since the previous token head does strange things (e.g., if you have an array of `[seq_len, d_model]`, it should become `[seq_len - 1, d_model]`).
  - Copy the token that follows the previous instance of the present token.
  - Use `attention_strength` for the non-zero entries in the QK matrices.
  - Use only 0's and 1's in the OV matrices.

### Output Format:
* Return an array of 4 logits (one for each vocabulary token)
* The output should be the prediction of the induction head on the last token in the sequence. The output should be just the output from that head (**not** the sum of residual stream with the head's output!).

### Important Implementation Notes
* **Causal masking:** When implementing attention, mask out positions that come after the current position by setting their attention scores to negative infinity (e.g., `-1e9`) before applying softmax.
* **Matrix construction:** Use `attention_strength` for non-zero entries in QK matrices, and use `1.0` for non-zero entries in OV matrices (not `attention_strength`)



### Example
For the sequence `[a, b, c, d, a]`, the prediction should upweight `b`.

#### Step-by-step for this example:
**Previous Token Head** creates outputs where position $i$ contains information about the token at $i - 1$.

After deleting position 0 and adding to token embeddings (without position info), we have representations that know both "what token is here" and "what token came before"

**Induction Head** at the final position sees token `a` and looks for where else `a` appeared with its predecessor information

It finds that position 0 had token `a` (with no predecessor)

It copies what came after position 0, which is token `b`

The output logits should thus have the highest value for token `b`

### Note:
* You should your use your previously implemented function `single_attention_head` for this problem.

### Hints
- Build the previous token head and the copy head separately; test each intermediate tensor to sanity-check shapes.
- Use the positional slots (dims 4-8) to aim queries at the previous position, and the token slots (dims 0-3) to copy token identities.
- When adding the residual stream, drop position 0 so the induction head only sees valid previous-token pairs.


In [6]:
def induction_copy_head(embeddings, attention_strength):
  """
  Implement an induction head mechanism combining previous token head and copying.

  Args:
      embeddings: List[List[float]] - Array of 2-hot encoding embeddings (token + position)
      attention_strength: float - Attention strength

  Returns:
      numpy.ndarray - Logits for the next token prediction (size 4)
  """
  # step -1: convert to numpy array
  X = np.asarray(embeddings)

  seq_len, d_model = X.shape
  vocab_size = 4

  # step 0: previous token head
  WQK_prev = np.zeros((d_model, d_model))
  WOV_prev = np.zeros((d_model, d_model))
  WQK_prev[vocab_size : vocab_size + seq_len, vocab_size: vocab_size + seq_len] = attention_strength * np.eye(seq_len, k=-1)
  WOV_prev[vocab_size : 2 * vocab_size, : vocab_size] = np.eye(vocab_size)  # IMPORTANT: write to dims 4-7 that don't overlap with original tokens (dims 0-3)
  X1 = single_attention_head(X, WQK_prev, WOV_prev)

  # step 0.5: add from residual stream
  X1[:seq_len, :vocab_size] += X[:seq_len, :vocab_size]  # exclude positional embeddings from sum
  X1 = X1[1:, :]

  # step 1: copy head
  WQK_copy = np.zeros((d_model, d_model))
  WOV_copy = np.zeros((d_model, d_model))
  WQK_copy[: vocab_size, vocab_size : 2 * vocab_size] = attention_strength * np.eye(vocab_size)  # IMPORTANT: match current token (query dims 0-3) with previous token (dims key 4-7)
  WOV_copy[: vocab_size, : vocab_size] = np.eye(vocab_size)

  X2 = single_attention_head(X1, WQK_copy, WOV_copy)

  # step 2: obtain logits
  out = X2[-1, :vocab_size]
  return out

### Test `induction_copy_head` basic

In [7]:
embeddings = [[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
attention_strength = 10.0

out = induction_copy_head(embeddings, attention_strength).tolist()

expected_out = [0.000045, 0.999864, 0.000045, 0.000045]

assert np.isclose(expected_out, out, atol=eps).all(), f"Failed:\nExpected: {expected_out}\nGot: {out}"
print("Test case passed ✅")

Test case passed ✅


### Test `induction_copy_head`

In [9]:
with open('induction_head_test_cases.json', 'r') as f:
  induction_head_test_cases = json.load(f)

for test_case_id in induction_head_test_cases:
  embeddings, attention_strength, expected_out = induction_head_test_cases[test_case_id]
  out = induction_copy_head(embeddings, attention_strength).tolist()
  id = test_case_id.split(" ")[-1]
  assert np.isclose(expected_out, out, atol=eps).all(), f"Test Case {id} Failed:\nExpected: {expected_out}\nGot: {out}"

print("All test cases passed ✅")

All test cases passed ✅


In [10]:
# # How to generate the test cases
# seq_lens = [2, 3, 4, 5]
# attention_strengths = [10, 20, 25, 40]
# vocab_size, max_seq_len = 4, 5
# i = 1
# out = {}

# for seq_len in seq_lens:
#   for attention_strength in attention_strengths:
#     embeddings = np.concatenate((np.eye(vocab_size)[np.random.randint(0, vocab_size, size=seq_len)], np.eye(max_seq_len)[: seq_len]), axis=1).tolist()
#     out[f"Test case {i}"] = [embeddings, attention_strength, induction_copy_head(embeddings, attention_strength).tolist()]
#     i += 1

# with open('induction_head_test_cases.json', 'w') as f:
#   json.dump(out, f, indent=2)